In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from statsmodels.tsa.vector_ar.var_model import VARProcess
from statsmodels.tsa.vector_ar.util import varsim


In [ ]:
def base_m_size_structure(lags, nvar, mb_size, seed=0):
    """
    Create a VAR with `lags` lags, nvar variables, and identical markov boundary size mb_size.
    The first variable should be the target.
    """
    random_generator = np.random.default_rng(seed)
    
    flag = False
    matrix = np.zeros((lags, nvar, nvar))
    for target in range(nvar):
        candidate = list(range(nvar))
        if target in candidate: candidate.remove(target)
        for k in range(mb_size):
            source = random_generator.choice(candidate)
            candidate.remove(source)
            lag = random_generator.choice(range(0,lags))
            matrix[lag, target, source] = (0.5+random_generator.random()*0.4) * random_generator.choice([-1,1])
        # big multilagged autoregressive term
        for lag in range(lags):
            matrix[lag, target, target] = (0.5+random_generator.random()*0.4) * random_generator.choice([-1,1])
    while not flag:
        matrix = matrix*(random_generator.random()*0.47+0.5)
        process = VARProcess(matrix, None, np.identity(nvar))
        flag = process.is_stable()
    return VARProcess(matrix, None, np.identity(nvar))

In [ ]:
def base_markov_boundary(var_names, process):
    """
    Given a VARProcess instance, and a list of target variables `var_names`, output triplets (source, target, lag).
    """
    all_relations = []
    for var in var_names:
        matrix = process.coefs
        matrix = matrix[:, var, :] != 0.
        lags, v = matrix.nonzero()
        for i in range(len(v)):
            source,effect,lag = v[i], var, lags[i]+1 # lag direction from source code
            all_relations.append((source,effect,lag))
    return all_relations

In [ ]:
def generate_theoretical_split(remaining, partition_size,
                               random_generator,
                               limit_size_1 = False):
    
    if remaining==0:return []
    replace = remaining<partition_size
    flag = True
    while flag:
        split_indices = [0]+sorted(random_generator.choice(list(range(1,remaining+1)),size=partition_size,replace=replace))
        split_indices = np.diff(split_indices)
        flag = 1 in split_indices if limit_size_1 else False
    sizes = list(split_indices)
    
    return sizes



def generation_function(lags,
                        nsamples,
                        mb_size,
                        total_irreplaceable_variables,
                        total_replaceable_variables,
                        total_redundant_variables,
                        total_irrelevant_variables,
                        fill_redundant=10,
                        seed=0):
    
    random_generator = np.random.default_rng(seed)
    
    # set the size of each equivalence class
    sizes = [1]*total_irreplaceable_variables  #size of each equivalent set
    remaining = total_replaceable_variables  #number of wanted copies
    partition_size = mb_size-total_irreplaceable_variables  # number of original variables to copy
    sizes = sizes + generate_theoretical_split(remaining, partition_size, random_generator, limit_size_1=True)
    
    
    # set the number of copies of each redundant variable
    remaining = total_redundant_variables
    partition_size = fill_redundant
    redundant_sizes = generate_theoretical_split(remaining, partition_size, random_generator)
    
    
    # create the base VAR
    nvar = 1+mb_size+fill_redundant
    base_var = base_m_size_structure(lags, nvar, mb_size, seed=seed)
    # get parents
    tpls = base_markov_boundary([0], base_var)
    parentindexlist = list(set([(s,l) for s,_,l in tpls if s!=0]))
    random_generator.shuffle(parentindexlist)
    
    # simulate VAR
    initial_values = random_generator.random(size=(lags,nvar))
    data = base_var.simulate_var(steps=nsamples+lags, seed=seed, initial_values=initial_values)
    
    
    theoretical_solution = [[0]]
    columns = {0:data[:,0]}
    # duplicate variables that are in the markov boundary
    idx = 1
    for i,size in enumerate(sizes):
        initial_var = parentindexlist[i][0]
        initial_lag = parentindexlist[i][1]
        theoretical_solution.append([])
        for s in range(size):
            columns[idx] = data[:,initial_var]+random_generator.normal(scale=0.1, size=(data[:,initial_var].shape))
            theoretical_solution[-1].append(idx)
            if s>0: #change lag uniformly between the lag itself and lag 1
                shiftforward = random_generator.integers(0,initial_lag-1,endpoint=True)
                columns[idx] = np.roll(columns[idx],shiftforward)
            idx+=1
            
    # duplicate redundant variables randomly from the remaining variables (fill_redundant parameter)
    remaining_base_variables = list(range(1,nvar))
    for v,_ in parentindexlist: remaining_base_variables.remove(v)
    for i,size in enumerate(redundant_sizes):
        initial_var = remaining_base_variables[i]
        for s in range(size):
            columns[idx] = data[:,initial_var]+random_generator.normal(scale=0.1, size=(data[:,initial_var].shape))
            idx+=1
    
    # add irrelevant variables
    remaining_irrelevant = total_irrelevant_variables
    while remaining_irrelevant!=0:
        i_nvar, i_mb_size = min(remaining_irrelevant,20),min(remaining_irrelevant-1,10)
        irrelevant_var = base_m_size_structure(lags, i_nvar, i_mb_size, seed=seed+remaining_irrelevant)
        initial_values = random_generator.random(size=(lags,i_nvar))
        irrelevant_data = irrelevant_var.simulate_var(steps=nsamples+lags, seed=seed+remaining_irrelevant, initial_values=initial_values)
        for i in range(i_nvar):
            columns[idx] = irrelevant_data[:,i]
            idx+=1
        remaining_irrelevant-=i_nvar
    
    df = pd.DataFrame.from_dict(columns)
    df = df.iloc[lags:]  # remove the first `lags` coeffs due to time shifts in equivalences.
    
    # shuffle columns
    df = df[random_generator.permutation(df.columns)]
    
    return df, theoretical_solution

In [ ]:
df, th = generation_function(10,3500,10,3,20,30,100,seed=0)

In [ ]:
th

# Generate the whole dataset

In [ ]:
characteristics = []
num_dataset=0

for lags in [1,5,10]:
    for nsamples in [9000]:
        for total_variables in [10, 100, 1000]:
            for mb_size in [1, 4, 9]:
                print(lags,nsamples,total_variables,mb_size)
                
                total_irreplaceable_variables = mb_size//3
                base_replaceable_variables = mb_size - total_irreplaceable_variables
                factor = {10:2,100:5,1000:20}[total_variables]
                total_replaceable_variables = factor*base_replaceable_variables
                total_redundant_variables = (total_variables - 1 - total_irreplaceable_variables - total_replaceable_variables)//3
                total_irrelevant_variables = total_variables - 1 - total_irreplaceable_variables - total_replaceable_variables - total_redundant_variables
                
                # the size 10 case needs special attention to proportions
                if total_variables==10 and mb_size==9:
                    total_irreplaceable_variables = 9
                    total_replaceable_variables = 0
                    total_redundant_variables = 0
                    total_irrelevant_variables = 0
                
                
                for seedadd in range(10):
                    df, th = generation_function(lags,
                                                 nsamples,
                                                 mb_size,
                                                 total_irreplaceable_variables,
                                                 total_replaceable_variables,
                                                 total_redundant_variables,total_irrelevant_variables,
                                                 fill_redundant=20,
                                                 seed=lags+(nsamples+total_variables+mb_size+seedadd)*10)

                    row = {"lags":lags,"nsamples":nsamples,"total_variables":total_variables,
                           "mb_size":mb_size,"total_irreplaceable_variables":total_irreplaceable_variables,
                           "total_replaceable_variables":total_replaceable_variables,
                           "total_redundant_variables":total_redundant_variables,
                           "total_irrelevant_variables":total_irrelevant_variables,
                           "fill_redundant":20,"seed":lags+(nsamples+total_variables+mb_size+seedadd)*10,
                           "theoretical_solution":str(th)}
                    row["filename"] = "data_"+str(num_dataset)+".csv"
                    characteristics.append(row)

                    num_dataset+=1
                    
                    df.to_csv("./NoisyVAR/"+row["filename"],index=False, compression="gzip")
                    
df = pd.DataFrame(characteristics)
df.to_csv("NoisyVAR_characteristics.csv")